<a href="https://colab.research.google.com/github/surjit84/ApacheBeamHandsOn/blob/main/Coder_classes_%26_Type_Safety_using_Type_Hint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 41.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 17.6 MB/s

In [2]:
from apache_beam import coders

In [3]:
coders.registry.get_coder(int)

VarIntCoder

In [4]:
coders.registry.get_coder(str)

StrUtf8Coder

Overide default coders

In [5]:
coders.registry.register_coder(int, coders.FloatCoder)

In [6]:
coders.registry.get_coder(int)

FloatCoder

Custom Coder-> Check in Github link: https://github.com/surjit84/apache-beam/tree/master/sdks/python/apache_beam/coders

Type Safety using Type Hints

In [11]:
import apache_beam as beam

p = beam.Pipeline()

events = (p
          |beam.Create(['one','two','three'])
          |beam.Filter(lambda x: x%2==0).with_input_types(int)
          )
p.run()

TypeCheckError: ignored

Use decorators

In [12]:
import apache_beam as beam

p = beam.Pipeline()

@beam.typehints.with_input_types(int)
class FilterValues(beam.DoFn):
  def process(self, element):
    if element % 2 ==0:
      yield element

events = (p
          |beam.Create(['one','two','three'])
          |beam.ParDo(FilterValues())
          )
p.run()

TypeCheckError: ignored

Type Hint with Custom Coder

In [ ]:
import apache_beam as beam
import typing

class Employee(object):
  def __init__(self, id, name):
    self.id = id
    self.name = name

class EmployeeCoder(beam.coders.Coder):

  def encode(self, employee):
    return ('%s:%s' % (employee.id, employee.name)).encode('utf-8')

  def decode(self, s):
    return Employee(*s.decode('utf-8').split(':'))

  def is_deterministic(self):
    return True

beam.coders.registry.register_coder(Employee, EmployeeCoder)

def split_file(input):
  name, id, salary = input.split(',')
  return Employee(id, name), int(salary)

result = (
    p
	| beam.io.ReadFromText('data.txt')
    | beam.Map(split_file)
    | beam.CombinePerKey(sum).with_input_types(typing.Tuple[Employee, int])
	)

p.run()